In [1]:
print("Sauvakävelyn data-analyysi - IMU-datan esikäsittely ja muunto tensoreiksi")

Sauvakävelyn data-analyysi - IMU-datan esikäsittely ja muunto tensoreiksi


In [2]:
import numpy as np # matematiikkakirjasto
import pandas as pd # datan käsittelykirjasto
import matplotlib.pyplot as plt # graafien piirtokirjasto
import torch # koneoppimiskirjasto muunnokseen tensoreiksi
from scipy.spatial.transform import Rotation as R
from ahrs.filters import Madgwick # IMU-datan esikäsittelykirjasto

In [3]:
df1 = pd.read_csv('data/imu_skdata2_031125.csv') # tehokas sauvakävely datanäyte

In [4]:
df1.head() # esitetään sensoridata dataframessa

,Timestamp,AccX,AccY,AccZ,GyroX,GyroY,GyroZ
0,475455,-5.999021,13.213638,0.581477,-80.919998,-157.289993,-18.480000
1,475531,-4.407737,11.232310,4.905462,-47.670002,-82.949997,-46.270000
2,475605,-5.514290,8.928150,0.547976,-9.300000,-2.730000,-81.760002
3,475680,-3.917191,4.424487,0.239291,-42.630001,8.610000,-55.510002
4,475756,-3.228033,4.864782,0.665228,-49.630001,-11.550000,-21.770000


In [31]:
# ---------------------------------------------------------
# 1. ALKUVALMISTELU
# ---------------------------------------------------------
# Lasketaan näytteenottotaajuus (Hz) keskimääräisestä aikaerosta. Timestamp on millisekunteina.
time_diffs = df1['Timestamp'].diff().dropna()
avg_dt_ms = time_diffs.mean() 
freq = 1000.0 / avg_dt_ms # Hz
print(f"Laskettu taajuus: {freq:.2f} Hz")

# Erotellaan data matriiseiksi (Samples x 3)
acc = df1[['AccX', 'AccY', 'AccZ']].values
gyro = df1[['GyroX', 'GyroY', 'GyroZ']].values

Laskettu taajuus: 13.37 Hz


In [32]:
# ---------------------------------------------------------
# 2. FUUSIO (Lasketaan orientaatio eli Quaternionit)
# ---------------------------------------------------------
# Muunnetaan Gyro asteista radiaaneiksi (Madgwick vaatii rad/s)
gyro_rad = np.deg2rad(gyro)
# Ahrs-kirjasto laskee Q:n automaattisesti koko datalle, 
# kun annamme 'gyr' ja 'acc' parametrit heti luonnin yhteydessä.
madgwick = Madgwick(gyr=gyro_rad, acc=acc, frequency=freq, beta=0.1)
# Valmis kvaterniomatriisi (N, 4)
Q = madgwick.Q
print("Kvaterniomatriisi:")
print(Q.shape)
print(Q)
# Joskus ahrs palauttaa Q:n joka on yhden pidempi tai lyhyempi riippuen versiosta,
# varmistetaan että pituus täsmää kiihtyvyysdataan:
if len(Q) != len(acc):
    # Yleensä Q voi olla 1 pidempi (alkutila), leikataan ylimääräinen pois
    Q = Q[:len(acc)]

# Q:n epätarkka määritys ilman ahrs-kirjastoa
# Q = np.array([[1.0, 0.0, 0.0, 0.0]] * len(acc))

Kvaterniomatriisi:
(151, 4)
[[ 7.06169934e-01  6.75777769e-01  1.52661918e-01 -1.46091649e-01]
 [ 7.32635222e-01  6.36075859e-01  1.37677857e-01 -1.99243422e-01]
 [ 7.21351006e-01  6.25973660e-01  1.74508371e-01 -2.39492238e-01]
 [ 7.24617707e-01  6.02406726e-01  2.11980791e-01 -2.59035633e-01]
 [ 7.40411678e-01  5.77989654e-01  2.20515266e-01 -2.62852665e-01]
 [ 7.58234440e-01  5.50229736e-01  2.31043709e-01 -2.62576801e-01]
 [ 7.60036553e-01  5.44145493e-01  2.36006808e-01 -2.65614207e-01]
 [ 7.62433957e-01  5.42209546e-01  2.32634915e-01 -2.65676996e-01]
 [ 7.63860890e-01  5.51572583e-01  2.16918702e-01 -2.55402630e-01]
 [ 7.56263302e-01  5.78584588e-01  1.97602694e-01 -2.32935331e-01]
 [ 7.48718278e-01  6.06000569e-01  1.74816455e-01 -2.04018279e-01]
 [ 7.58408004e-01  6.16663309e-01  1.35492883e-01 -1.61818854e-01]
 [ 7.88904107e-01  6.03347847e-01  6.46721612e-02 -9.70525480e-02]
 [ 8.09603813e-01  5.83438836e-01 -6.32190396e-02  1.20059970e-02]
 [ 7.99873028e-01  5.06833734e-01 

In [33]:
# ---------------------------------------------------------
# 3. KOORDINAATTIMUUNNOS (World Frame & Painovoiman poisto)
# ---------------------------------------------------------
# Luodaan rotaatio-objekti kvaternioista
r = R.from_quat(Q[:, [1, 2, 3, 0]]) # Scipy käyttää järjestystä [x, y, z, w], ahrs [w, x, y, z]

# Käännetään kiihtyvyysvektorit globaaliin koordinaatistoon
acc_world = r.apply(acc)

# Poistetaan painovoima (oletetaan Z-suuntaan 9.81 m/s^2)
gravity = np.array([0, 0, 9.81])
lin_acc_world = acc_world - gravity

# Yhdistetään uudet "cleanit" featuret (LinAcc + Gyro)
# Nyt meillä on 6 kanavaa: AccX_world, AccY_world, AccZ_world, GyroX, GyroY, GyroZ
# Käytetään raakaa gyroa (rad/s) mutta kiihtyvyys on "puhdistettu".
final_data = np.hstack((lin_acc_world, gyro_rad))

In [34]:
# ---------------------------------------------------------
# 4. IKKUNOINTI (Sliding Window)
# ---------------------------------------------------------
# Ikkunan koko ja limitys. Näyte pilkotaan kahden sekunnin mittaisiin aika-ikkunoihin.
WINDOW_SECONDS = 2.0
STRIDE_SECONDS = 1.0 # 50% limitys

window_size = int(WINDOW_SECONDS * freq)
stride = int(STRIDE_SECONDS * freq)

def create_windows(data, window_size, step):
    # data: (Samples, Channels)
    # Output: (Num_Windows, Window_Size, Channels)
    n_samples, n_channels = data.shape
    # Lasketaan ikkunoiden määrä
    n_windows = (n_samples - window_size) // step + 1
    
    if n_windows <= 0:
        return np.empty((0, window_size, n_channels))
        
    # Numpy-temppu muistin säästämiseksi (view, ei copy)
    shape = (n_windows, window_size, n_channels)
    strides = (step * data.strides[0], data.strides[0], data.strides[1])
    
    return np.lib.stride_tricks.as_strided(data, shape=shape, strides=strides)

windows = create_windows(final_data, window_size, stride)

print(f"Ikkunoiden muoto (N, T, C): {windows.shape}")

Ikkunoiden muoto (N, T, C): (10, 26, 6)


In [35]:
# ---------------------------------------------------------
# 5. TENSORIKSI (PyTorch)
# ---------------------------------------------------------
# Muutetaan lopulliseen muotoon (float32)
tensor = torch.from_numpy(windows).float()


print("Tensorin muoto:", tensor.shape)
print("Tensorin tyyppi:", tensor.dtype)
torch.set_printoptions(threshold=10_000)
print("\nValmis Tensori printattuna:")
print(tensor) 

Tensorin muoto: torch.Size([10, 26, 6])
Tensorin tyyppi: torch.float32

Valmis Tensori printattuna:
tensor([[[ 4.5797e-16, -1.8874e-15,  4.7133e+00, -1.4123e+00, -2.7452e+00,
          -3.2254e-01],
         [ 1.1021e+00, -3.0748e+00,  2.7990e+00, -8.3200e-01, -1.4478e+00,
          -8.0756e-01],
         [ 4.6318e-01,  1.0670e+00,  6.3349e-01, -1.6232e-01, -4.7647e-02,
          -1.4270e+00],
         [-2.4966e-01,  8.5439e-01, -3.9632e+00, -7.4403e-01,  1.5027e-01,
          -9.6883e-01],
         [ 6.8071e-01,  7.2930e-01, -4.0192e+00, -8.6621e-01, -2.0159e-01,
          -3.7996e-01],
         [-4.1182e-01,  1.8994e+00, -2.3984e+00, -1.0091e+00, -1.9059e-01,
          -1.5149e-01],
         [-1.1903e+00,  1.8229e+00, -1.5977e+00, -2.5168e-01, -1.0821e-01,
          -7.5747e-02],
         [-1.7543e+00,  1.9069e+00, -2.3878e+00, -1.2584e-01, -2.1136e-01,
           1.5394e-01],
         [-1.5460e+00,  1.3114e+00, -2.9907e+00,  1.4416e-01, -2.2602e-01,
           6.0842e-01],
         